# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 116234127,
 'calculationPrice': 'tops',
 'change': 3.96,
 'changePercent': 0.02247,
 'close': 0,
 'closeSource': 'ofliifac',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': ' miXtlieIr pe crEae',
 'highTime': 1655095265361,
 'iexAskPrice': 188,
 'iexAskSize': 109,
 'iexBidPrice': 177,
 'iexBidSize': 112,
 'iexClose': 183.38,
 'iexCloseTime': 1686984985014,
 'iexLastUpdated': 1674397006535,
 'iexMarketPercent': 0.011188261551556016,
 'iexOpen': 177.677,
 'iexOpenTime': 1647013455739,
 'iexRealtimePrice': 184.01,
 'iexRealtimeSize': 103,
 'iexVolume': 952160,
 'lastTradeTime': 1644245182101,
 'latestPrice': 182.67,
 'latestSource': 'IEX real time price',
 'latestTime': '3:02:08 PM',
 'latestUpdate': 1653769097843,
 'latestVolume': None,
 'low': 0,
 'lowSource': None

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
pe_ratio = data['peRatio']
pe_ratio

16.13

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'KSU':
            continue
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,155.930,48.2,N/A
1,AAL,17.151,-3.37,N/A
2,AAP,243.160,24.86,N/A
3,AAPL,184.300,16.59,N/A
4,ABBV,135.810,31.63,N/A
...,...,...,...,...
499,YUM,138.400,26.3,N/A
500,ZBH,123.340,30.93,N/A
501,ZBRA,590.420,39.42,N/A
502,ZION,63.980,5.94,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,HPE,15.161,2.86,13191
1,BIO,750.970,3.35,266
2,COO,409.910,3.45,487
3,HPQ,36.110,3.5,5538
4,EBAY,65.500,3.59,3053
5,NRG,40.600,4.03,4926
6,MPC,64.010,4.34,3124
7,DHI,105.760,4.7,1891
8,BEN,35.200,4.88,5681
9,TSN,88.440,5.2,2261


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'KSU':
            continue
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [13]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,187.900,N/A,59.47,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,303.140,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,306.200,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,12.000,N/A,10.18,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,21.440,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.000,N/A,14.5,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,59.110,N/A,32.5,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,33.360,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2971.000,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
324,MXIM,103.190,N/A,34.34,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [14]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [16]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.821429
1      0.051587
2      0.511905
3      0.305556
4      0.636905
         ...   
499    0.549603
500    0.634921
501    0.742063
502    0.095238
503    0.865079
Name: PE Percentile, Length: 504, dtype: object
0      0.805556
1      0.055556
2       0.60119
3      0.938492
4      0.910714
         ...   
499    0.051587
500    0.301587
501     0.84127
502    0.157738
503    0.946429
Name: PB Percentile, Length: 504, dtype: object
0      0.818452
1      0.041667
2      0.193452
3      0.611111
4      0.614087
         ...   
499    0.761905
500    0.525794
501    0.751984
502    0.450397
503    0.928571
Name: PS Percentile, Length: 504, dtype: object
0       0.85119
1       0.02381
2      0.468254
3      0.402778
4      0.343254
         ...   
499    0.718254
500    0.613095
501    0.827381
502     0.06746
503    0.934524
Name: EV/EBITDA Percentile, Length: 504, dtype: object
0       0.85119
1      0.063492
2      0.180556
3      0.674603
4      0.551587
         ...   
4

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,157.94,N/A,47.70,0.821429,9.70,0.805556,7.6000,0.818452,30.886959,0.85119,14.625695,0.85119,N/A
1,AAL,17.44,N/A,-3.40,0.051587,-1.51,0.055556,0.4573,0.041667,-7.150533,0.02381,1.422469,0.063492,N/A
2,AAP,245.08,N/A,24.59,0.511905,4.72,0.60119,1.3400,0.193452,13.718076,0.468254,3.109899,0.180556,N/A
3,AAPL,181.39,N/A,15.99,0.305556,23.24,0.938492,4.1000,0.611111,12.423630,0.402778,9.729603,0.674603,N/A
4,ABBV,136.47,N/A,31.17,0.636905,16.80,0.910714,4.1100,0.614087,10.914097,0.343254,7.800230,0.551587,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,135.60,N/A,26.00,0.549603,-4.99,0.051587,6.0400,0.761905,22.180430,0.718254,10.410828,0.708333,N/A
500,ZBH,123.24,N/A,31.11,0.634921,2.08,0.301587,3.3200,0.525794,18.059951,0.613095,5.627888,0.380952,N/A
501,ZBRA,617.33,N/A,38.53,0.742063,11.45,0.84127,5.7500,0.751984,28.711780,0.827381,12.359824,0.78373,N/A
502,ZION,64.26,N/A,6.09,0.095238,1.37,0.157738,2.8600,0.450397,4.767414,0.06746,2.542102,0.128968,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [17]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,157.94,N/A,47.70,0.821429,9.70,0.805556,7.6000,0.818452,30.886959,0.85119,14.625695,0.85119,0.829563
1,AAL,17.44,N/A,-3.40,0.051587,-1.51,0.055556,0.4573,0.041667,-7.150533,0.02381,1.422469,0.063492,0.047222
2,AAP,245.08,N/A,24.59,0.511905,4.72,0.60119,1.3400,0.193452,13.718076,0.468254,3.109899,0.180556,0.391071
3,AAPL,181.39,N/A,15.99,0.305556,23.24,0.938492,4.1000,0.611111,12.423630,0.402778,9.729603,0.674603,0.586508
4,ABBV,136.47,N/A,31.17,0.636905,16.80,0.910714,4.1100,0.614087,10.914097,0.343254,7.800230,0.551587,0.61131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,135.60,N/A,26.00,0.549603,-4.99,0.051587,6.0400,0.761905,22.180430,0.718254,10.410828,0.708333,0.557937
500,ZBH,123.24,N/A,31.11,0.634921,2.08,0.301587,3.3200,0.525794,18.059951,0.613095,5.627888,0.380952,0.49127
501,ZBRA,617.33,N/A,38.53,0.742063,11.45,0.84127,5.7500,0.751984,28.711780,0.827381,12.359824,0.78373,0.789286
502,ZION,64.26,N/A,6.09,0.095238,1.37,0.157738,2.8600,0.450397,4.767414,0.06746,2.542102,0.128968,0.17996


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [18]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [19]:
portfolio_input()

Enter the value of your portfolio:10000000


In [20]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

C:\ProgramData\Anaconda3\envs\algorithm_trading\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.820,8396,6.18,0.097222,0.4450,0.05754,0.3816,0.03373,2.962943,0.031746,0.362268,0.007937,0.045635
1,AAL,17.440,11467,-3.40,0.051587,-1.5100,0.055556,0.4573,0.041667,-7.150533,0.02381,1.422469,0.063492,0.047222
2,HPQ,36.990,5406,3.47,0.06746,-12.2100,0.037698,0.3050,0.013889,3.881308,0.045635,1.738934,0.085317,0.05
3,PRU,108.321,1846,5.82,0.089286,0.6759,0.065476,0.6003,0.055556,4.405363,0.055556,0.555011,0.009921,0.055159
4,BEN,34.660,5770,4.80,0.078373,0.7694,0.079365,1.0500,0.138889,3.316980,0.035714,0.861632,0.019841,0.070437
5,HPE,15.120,13227,2.89,0.063492,0.4916,0.059524,0.3434,0.027778,7.095569,0.150794,1.921630,0.099206,0.080159
6,BA,199.900,1000,-13.39,0.041667,-7.9800,0.047619,1.8900,0.303571,-31.317330,0.005952,-1288.204103,0.001984,0.080159
7,KSS,50.940,3926,7.85,0.125,1.5000,0.18254,0.3711,0.029762,3.817750,0.041667,1.123965,0.043651,0.084524
8,AIG,56.160,3561,8.80,0.140873,0.7120,0.071429,0.9830,0.126984,4.433897,0.05754,0.962234,0.029762,0.085317
9,MET,61.040,3276,10.39,0.174603,0.7458,0.075397,0.7442,0.085317,5.203401,0.085317,0.748180,0.017857,0.087698


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [21]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [22]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [23]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [24]:
writer.save()